In [1]:
%%capture
# run this cell if this package is not installed
# !pip install nbformat

In [2]:
%%capture
%run setup.ipynb

In [11]:
import nltk
nltk.download('words')
from nltk.corpus import words

english_word_set = set(words.words())

def is_english_word(word):
    return word.lower() in english_word_set

In [91]:
def normalize_document(doc):
    wpt = nltk.WordPunctTokenizer()
    stop_words = set(nltk.corpus.stopwords.words('english'))

    # sentences to be removed
    p1 = "Chat Conversation Start"
    p2 = "Chat Conversation End"
    
    # Remove p1 only the first time it appears
    doc = re.sub(rf'^\s*{re.escape(p1)}\s*', '', doc)

    # Remove p2 only the last time it appears
    doc = re.sub(rf'\s*{re.escape(p2)}\s*$', '', doc)
    
    # remove email addresses
    doc = re.sub(r'\b\S*@\S*\.\S*\b', '', doc)
    
    # remove special characters and digits, retaining only words with letters
    doc = re.sub(r'[^\w\s]', '', doc)
    
    # lowercase and strip
    doc = doc.lower()
    doc = doc.strip()
    
    # remove brackets of any kind
    doc = re.sub(r'[(){}[\]]', '', doc)
    
    # remove punctuation
    doc = doc.translate(str.maketrans("", "", string.punctuation))

    # retain only English words
    doc = ' '.join(word for word in doc.split() if is_english_word(word))
    
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    
    return doc



In [92]:
df_norm = df['Content'].apply(normalize_document)

In [94]:
df_norm.iloc[1000]

'subject developable surface organization university hong tin version hi currently developable surface anyone familiar topic give information allow find developable surface thanks help'

In [95]:
df['File_Name'].iloc[1000]

'38676.txt'

In [87]:
# %%capture
# # spacy.cli.download("en_core_web_sm")

# nlp = spacy.load("en_core_web_sm")

In [88]:
# def lemmatizer(text):        
#     sent = []
#     doc = nlp(text)
#     #print(doc)
#     for word in doc:
#         sent.append(word.lemma_)
#     return " ".join(sent)

In [96]:
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(df_norm)
cv_matrix

<11336x21253 sparse matrix of type '<class 'numpy.int64'>'
	with 778402 stored elements in Compressed Sparse Row format>

In [97]:
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [98]:
# get all unique words in the corpus
vocab = cv.get_feature_names_out()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

,aa,abacus,abandon,abandoned,abandonment,abate,abatement,abbey,abbot,abbreviation,...,zombie,zone,zoning,zoo,zooid,zoological,zoology,zoom,zorro,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11331,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11332,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11333,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11334,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
from sklearn.feature_extraction.text import TfidfTransformer

tt = TfidfTransformer(norm='l2', use_idf=True)
tt_matrix = tt.fit_transform(cv_matrix)


tt_matrix = tt_matrix.toarray()
vocab = cv.get_feature_names_out()
pd.DataFrame(np.round(tt_matrix, 2), columns=vocab)

,aa,abacus,abandon,abandoned,abandonment,abate,abatement,abbey,abbot,abbreviation,...,zombie,zone,zoning,zoo,zooid,zoological,zoology,zoom,zorro,zucchini
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
